# Raspagem e preparação dos dados

A etapa inicial do projeto é a obtenção dos dados do Twitter, faremos o scraping (raspagem) dos dados, para isso utilizaremos de duas fontes, um conjunto de dados presente no Kraggle e um conjunto de dados obtido através da API do Twitter.

In [2]:
import pandas as pd
import tweepy as tw
import pickle as pkl
import getpass

## Dados do Kraggle
O conjunto de dados do Kraggle _Jair Bolsonaro Twitter Data_ apresenta uma coleção de 3 mil tweets do Bolsonaro.

In [3]:
kraggle_data = pd.read_csv("..//data//tweets//bolsonaro_tweets.csv")
kraggle_data.date = pd.to_datetime(kraggle_data.date)
kraggle_data.head()

,date,text,likes,retweets,link
0,2020-08-01,- Ministro @tarcisiogdf e os 160 anos da Infra...,880,163,https://twitter.com/jairbolsonaro/status/12895...
1,2020-08-01,"- Coletiva com o prefeito de Bagé/RS, Divaldo ...",336,51,https://twitter.com/jairbolsonaro/status/12895...
2,2020-08-01,@rsallesmma 👍🏻,442,39,https://twitter.com/jairbolsonaro/status/12893...
3,2020-08-01,@Marceloscf2 @tarcisiogdf Seguimos! 🤝,164,22,https://twitter.com/jairbolsonaro/status/12893...
4,2020-08-01,@ItamaratyGovBr @USAID 👍🏻,223,24,https://twitter.com/jairbolsonaro/status/12893...


In [6]:
print("O período de " +  str(len(kraggle_data))+" tweets é de " +  str(kraggle_data.date.min())[0:10] +" até " +  str(kraggle_data.date.max())[0:10] + ".")

O período de 9351 tweets é de 2010-04-01 até 2020-08-01.


## Twitter API
Nós também podemos obter dados sobre Jair Bolsonaro, Flávio Bolsonaro, Carlos Bolsonaro e Eduardo Bolsonaro da API do Twitter usando Tweepy. No entanto, o acesso é permitido apenas para os últimos 3200 tweets, dessa forma, não é o histórico completo de tweets.

In [3]:
consumer_key = getpass.getpass()

········


In [4]:
consumer_secret = getpass.getpass()

········


In [5]:
access_token = getpass.getpass()

········


In [6]:
access_token_secret = getpass.getpass()

········


In [7]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [20]:
#requesting data from twitter
twitter_data = {}
for user in ['jairbolsonaro', 'bolsonarosp', 'flaviobolsonaro', 'carlosbolsonaro']:
    twitter_data[user] = [tweet for tweet in tw.Cursor(api.user_timeline,id=user, tweet_mode ="extended").items()]

In [22]:
#saving original data
with open("..//data//tweets//brute_scrapping,pkl", "wb+") as f:
    pkl.dump(twitter_data, f)

In [36]:
#script to create a dataframe from the data
date = []
text = []
user_name = []
place = []
retweets = []
favorites = []
for user in ['jairbolsonaro', 'bolsonarosp', 'flaviobolsonaro', 'carlosbolsonaro']:
    for tweet in twitter_data[user]:
        date.append(tweet._json['created_at'])
        text.append(tweet._json['full_text'])
        user_name.append(user)
        place.append(tweet._json['place'])
        retweets.append(tweet._json['retweet_count'])
        favorites.append(tweet._json['favorite_count'])
API_data = pd.DataFrame({'name': user_name, 'text': text, 'date': pd.to_datetime(date), 
                         'place': place, 'retweets': retweets, 'favorites':favorites})
API_data.head()

,name,text,date,place,retweets,favorites
0,jairbolsonaro,"-Edifício Joelma/SP, 1974.\n\n-Sgt CASSANIGA s...",2020-07-27 20:51:13+00:00,None,3154,16202
1,jairbolsonaro,- Água para quem tem sede.\n- Liberdade para u...,2020-07-27 11:10:36+00:00,None,8101,37357
2,jairbolsonaro,"@tarcisiogdf @MInfraestrutura 🤝🇧🇷, Ministro!",2020-07-26 20:18:19+00:00,None,1074,16840
3,jairbolsonaro,2- @MinEconomia @MinCidadania @onyxlorenzoni @...,2020-07-26 15:40:39+00:00,None,1337,6383
4,jairbolsonaro,1- Acompanhe as redes sociais! @secomvc @fabio...,2020-07-26 15:39:47+00:00,None,3287,14836


In [51]:
API_data.to_csv('..API_data.csv', sep = "~")